In [2]:
import tarfile
import glob
import shutil
import SimpleITK as sitk
import numpy as np
import os
from collections import Counter
from multiprocessing import Pool
from numpy.lib.npyio import save
from numpy.lib.shape_base import _dstack_dispatcher
from numpy.lib.type_check import imag
from pathlib import Path
import pathlib
import multiprocessing
import math
import functools
from functools import partial
import time
import tqdm
from multiprocessing import Pool
import os, time, random

In [3]:

def get_file(root_path,all_files=[]):
    '''
    递归函数，遍历该文档目录和子目录下的所有文件，获取其path
    '''
    files = os.listdir(root_path)
    for file in files:
        if not os.path.isdir(root_path + '/' + file):   # not a dir
            all_files.append(root_path + '/' + file)
        else:  # is a dir
            get_file((root_path+'/'+file),all_files)
    return all_files

# example
path = '/data/wyh/影像数据EGFR1第二部分'
all_path=get_file(path)


In [4]:
temp=list(filter(lambda x:Path(x).suffix=='.tar',all_path))#过滤掉那部分重合的文件夹，只对压缩包进行处理
temp=temp[1:]#这就是要处理的所有压缩包,第一个是已经解压过压缩包,基本上应该没有什么问题

In [5]:
temp

['/data/wyh/影像数据EGFR1第二部分/Marked/POSITIVE/3-1marked1.tar',
 '/data/wyh/影像数据EGFR1第二部分/primary_new/predict_PFS/1-22_1-23.tar',
 '/data/wyh/影像数据EGFR1第二部分/primary_new/EGFR-negative/6-4_6-4_NIFTI.tar',
 '/data/wyh/影像数据EGFR1第二部分/primary_new/EGFR-negative/6-10_6-10_NIFTI_6-12_6-12_NIFTI.tar',
 '/data/wyh/影像数据EGFR1第二部分/primary_new/EGFR-negative/4-10_4-11_4-17_4-21.tar',
 '/data/wyh/影像数据EGFR1第二部分/primary_new/EGFR-negative/6-11_6-11-0_6-11_NIFTI.tar',
 '/data/wyh/影像数据EGFR1第二部分/primary_new/EGFR-positive/20191116positive.tar',
 '/data/wyh/影像数据EGFR1第二部分/primary_new/EGFR-positive/2-25_2-25left_2-28_2-28left.tar',
 '/data/wyh/影像数据EGFR1第二部分/primary_new/EGFR-positive/3-1No.1_3-1No2_3-1No3_3-1No4_3-1No5_3-5.tar']

In [6]:

def str_insert(str_origin, pos=9, str_add='_data'):
    str_list = list(str_origin)    # 字符串转list
    str_list.insert(pos, str_add)  # 在指定位置插入字符串
    str_out = ''.join(str_list)    # 空字符连接
    return  str_out

In [7]:
def remove_suffix(path):
    path = Path(path)
    return path.with_suffix("")

In [8]:
object_path=list(map(partial(str_insert,pos=9,str_add='_data'),temp))#终于掌握了偏函数的用法，会自己用了，好开心，就是重新自定义函数的一个办法
#这也是臆测的，结果就成了，他这里应该传递的是一个带参的函数
object_path=list(map(remove_suffix,object_path))

In [9]:
object_path

[PosixPath('/data/wyh_data/影像数据EGFR1第二部分/Marked/POSITIVE/3-1marked1'),
 PosixPath('/data/wyh_data/影像数据EGFR1第二部分/primary_new/predict_PFS/1-22_1-23'),
 PosixPath('/data/wyh_data/影像数据EGFR1第二部分/primary_new/EGFR-negative/6-4_6-4_NIFTI'),
 PosixPath('/data/wyh_data/影像数据EGFR1第二部分/primary_new/EGFR-negative/6-10_6-10_NIFTI_6-12_6-12_NIFTI'),
 PosixPath('/data/wyh_data/影像数据EGFR1第二部分/primary_new/EGFR-negative/4-10_4-11_4-17_4-21'),
 PosixPath('/data/wyh_data/影像数据EGFR1第二部分/primary_new/EGFR-negative/6-11_6-11-0_6-11_NIFTI'),
 PosixPath('/data/wyh_data/影像数据EGFR1第二部分/primary_new/EGFR-positive/20191116positive'),
 PosixPath('/data/wyh_data/影像数据EGFR1第二部分/primary_new/EGFR-positive/2-25_2-25left_2-28_2-28left'),
 PosixPath('/data/wyh_data/影像数据EGFR1第二部分/primary_new/EGFR-positive/3-1No.1_3-1No2_3-1No3_3-1No4_3-1No5_3-5')]

In [10]:
temp

['/data/wyh/影像数据EGFR1第二部分/Marked/POSITIVE/3-1marked1.tar',
 '/data/wyh/影像数据EGFR1第二部分/primary_new/predict_PFS/1-22_1-23.tar',
 '/data/wyh/影像数据EGFR1第二部分/primary_new/EGFR-negative/6-4_6-4_NIFTI.tar',
 '/data/wyh/影像数据EGFR1第二部分/primary_new/EGFR-negative/6-10_6-10_NIFTI_6-12_6-12_NIFTI.tar',
 '/data/wyh/影像数据EGFR1第二部分/primary_new/EGFR-negative/4-10_4-11_4-17_4-21.tar',
 '/data/wyh/影像数据EGFR1第二部分/primary_new/EGFR-negative/6-11_6-11-0_6-11_NIFTI.tar',
 '/data/wyh/影像数据EGFR1第二部分/primary_new/EGFR-positive/20191116positive.tar',
 '/data/wyh/影像数据EGFR1第二部分/primary_new/EGFR-positive/2-25_2-25left_2-28_2-28left.tar',
 '/data/wyh/影像数据EGFR1第二部分/primary_new/EGFR-positive/3-1No.1_3-1No2_3-1No3_3-1No4_3-1No5_3-5.tar']

In [11]:
object_path

[PosixPath('/data/wyh_data/影像数据EGFR1第二部分/Marked/POSITIVE/3-1marked1'),
 PosixPath('/data/wyh_data/影像数据EGFR1第二部分/primary_new/predict_PFS/1-22_1-23'),
 PosixPath('/data/wyh_data/影像数据EGFR1第二部分/primary_new/EGFR-negative/6-4_6-4_NIFTI'),
 PosixPath('/data/wyh_data/影像数据EGFR1第二部分/primary_new/EGFR-negative/6-10_6-10_NIFTI_6-12_6-12_NIFTI'),
 PosixPath('/data/wyh_data/影像数据EGFR1第二部分/primary_new/EGFR-negative/4-10_4-11_4-17_4-21'),
 PosixPath('/data/wyh_data/影像数据EGFR1第二部分/primary_new/EGFR-negative/6-11_6-11-0_6-11_NIFTI'),
 PosixPath('/data/wyh_data/影像数据EGFR1第二部分/primary_new/EGFR-positive/20191116positive'),
 PosixPath('/data/wyh_data/影像数据EGFR1第二部分/primary_new/EGFR-positive/2-25_2-25left_2-28_2-28left'),
 PosixPath('/data/wyh_data/影像数据EGFR1第二部分/primary_new/EGFR-positive/3-1No.1_3-1No2_3-1No3_3-1No4_3-1No5_3-5')]

In [17]:
import time
from tqdm import tqdm, trange

def extract_tar(tar_path,out_path=[]):
    # print(tar_path)
    # print('******')
    # print(out_path)
    for i,j in tqdm(zip(tar_path,out_path)):
        tf = tarfile.open(i)
        tf.extractall(j)


if __name__=="__main__":
    pool = Pool(processes=10)
    # for i,j  in tqdm(zip(temp,object_path)):
    pool.apply(extract_tar, args=(temp,object_path))
    print('test')

9it [1:41:09, 674.43s/it]


test


In [16]:
# import time
# from tqdm import tqdm, trange

# def extract_tar(tar_path,out_path=[]):
#     print(tar_path)
#     print('******')
#     print(out_path)
#     for i,j in tqdm(zip(tar_path,out_path)):
#         tf = tarfile.open(str(i))
#         tf.extractall(str(j))


# # if __name__=="__main__":
# #     pool = Pool(processes=10)
# #     # for i,j  in tqdm(zip(temp,object_path)):
# #     pool.apply_async(extract_tar, args=(temp,object_path))
# #     print('test')

# # from multiprocessing import Pool

# if  __name__ == "__main__":
#     pool = Pool(processes=96)       #定义最大的进程数
#     pool.map(partial(extract_tar,out_path=object_path),temp)          #p必须是一个可迭代变量。
#     pool.close()
#     pool.join()


/data/wyh/影像数据EGFR1第二部分/Marked/POSITIVE/3-1marked1.tar/data/wyh/影像数据EGFR1第二部分/primary_new/EGFR-positive/20191116positive.tar/data/wyh/影像数据EGFR1第二部分/primary_new/predict_PFS/1-22_1-23.tar/data/wyh/影像数据EGFR1第二部分/primary_new/EGFR-negative/6-4_6-4_NIFTI.tar/data/wyh/影像数据EGFR1第二部分/primary_new/EGFR-negative/6-10_6-10_NIFTI_6-12_6-12_NIFTI.tar/data/wyh/影像数据EGFR1第二部分/primary_new/EGFR-negative/4-10_4-11_4-17_4-21.tar/data/wyh/影像数据EGFR1第二部分/primary_new/EGFR-negative/6-11_6-11-0_6-11_NIFTI.tar/data/wyh/影像数据EGFR1第二部分/primary_new/EGFR-positive/2-25_2-25left_2-28_2-28left.tar/data/wyh/影像数据EGFR1第二部分/primary_new/EGFR-positive/3-1No.1_3-1No2_3-1No3_3-1No4_3-1No5_3-5.tar






******
******
************************
******




******
[PosixPath('/data/wyh_data/影像数据EGFR1第二部分/Marked/POSITIVE/3-1marked1'), PosixPath('/data/wyh_data/影像数据EGFR1第二部分/primary_new/predict_PFS/1-22_1-23'), PosixPath('/data/wyh_data/影像数据EGFR1第二部分/primary_new/EGFR-negative/6-4_6-4_NIFTI'), PosixPath('/data/wyh_data/影像数据EGFR1第二部分/prima

0it [00:00, ?it/s]

0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]
0it [00:00, ?it/s]

0it [00:00, ?it/s]


0it [00:00, ?it/s]


IsADirectoryError: [Errno 21] Is a directory: '/'